# 7054-12-300-l_drucker_se_su_st_st_512_32

## Convert directory structure from synthnet evaluation pipeline format to Huggingface format

### Training Split

In [7]:
import os
import shutil
from datasets import load_dataset
# load a local dataset from image directory (huggingface)
# https://huggingface.co/docs/datasets/image_dataset
DATASET_TRAIN_NAME = 'visda2017'
DS_TRAIN_DIR = f'/home/dritter/projects/evaluation-pipeline/data/{DATASET_TRAIN_NAME}/train/'
OUT_ROOT_TRAIN = f'data/{DATASET_TRAIN_NAME}'
# DS_TRAIN_DIR = f'/home/dennis/Desktop/work/evaluation_pipeline_data/visda/train/{DATASET_TRAIN_NAME}'
# load the our data dir (evaluation pipeline format) 
# change directory structure 
#   from:   data/ds_name/images/class/mesh/images
#   to:     data/ds_name/images/class/images
for path, dns, fns in os.walk(DS_TRAIN_DIR):
    for fn in fns:
        split_path = path.split('/')
        label = split_path[-2]
        split = 'train'
        os.makedirs(f'{OUT_ROOT_TRAIN}/{label}', exist_ok=True)
        shutil.copy(f'{path}/{fn}', f'{OUT_ROOT_TRAIN}/{split}/{label}/{fn}')

# load cifar10 (only small portion for demonstration purposes) 
# train_ds, test_ds = load_dataset('cifar10', split=['train[:5000]', 'test[:2000]'])
# # split up training into training + validation
# splits = train_ds.train_test_split(test_size=0.1)
# train_ds = splits['train']
# val_ds = splits['test']

### Test Split

In [8]:
import os
import shutil
from datasets import load_dataset
# load a local dataset from image directory (huggingface)
# https://huggingface.co/docs/datasets/image_dataset
DATASET_TEST_NAME = 'visda2017'
DS_TRAIN_DIR = f'data/{DATASET_TRAIN_NAME}/train'
DS_TEST_DIR = f'/home/dritter/projects/evaluation-pipeline/data/datasets/{DATASET_TEST_NAME}/val'
OUT_ROOT_TEST = f'data/{DATASET_TEST_NAME}/val'

# Get classes present in train dataset
TRAIN_CLASSES = os.listdir('data/visda2017/train')
print(TRAIN_CLASSES)

for path, dns, fns in os.walk(DS_TEST_DIR):
    for fn in fns:
        split_path = path.split('/')
        label = split_path[-1]
        if label in TRAIN_CLASSES:
            os.makedirs(f'{OUT_ROOT_TEST}/{label}', exist_ok=True)
            shutil.copy(f'{path}/{fn}', f'{OUT_ROOT_TEST}/{label}/{fn}')

# load cifar10 (only small portion for demonstration purposes) 
# train_ds, test_ds = load_dataset('cifar10', split=['train[:5000]', 'test[:2000]'])
# # split up training into training + validation
# splits = train_ds.train_test_split(test_size=0.1)
# train_ds = splits['train']
# val_ds = splits['test']

['bicycle', 'horse', 'car', 'person', 'motorcycle', 'truck', 'aeroplane', 'skateboard', 'train', 'plant', 'bus', 'knife']


### Sanity check

In [17]:
DATASET_TRAIN_NAME = '7054-12-300-l_drucker_se_su_st_st_512_32'
DATASET_TEST_NAME = 'topex-real-123_pb_256'
TRAIN_CLASSES = os.listdir(f"data/{DATASET_TRAIN_NAME}")
TEST_CLASSES = os.listdir(f"data/{DATASET_TEST_NAME}")

for c in TEST_CLASSES:
    if c not in TRAIN_CLASSES:
        print(c)


In [13]:
from sklearn.model_selection import StratifiedShuffleSplit
import numpy as np



X = []
y =[]
for i in range(0,100):
    X.append(f"img_{i}")
    y.append(f"label_{i%5}")
print(X)
print(y)
X = np.array(X)
y = np.array(y)

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=42)
sss.get_n_splits(X, y)
for train_index, test_index in sss.split(X, y):
    print("TRAIN_i:", train_index)
    print("TEST_i:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
print("TRAIN", (X_train, y_train))
print("TEST", (X_test, y_test))

['img_0', 'img_1', 'img_2', 'img_3', 'img_4', 'img_5', 'img_6', 'img_7', 'img_8', 'img_9', 'img_10', 'img_11', 'img_12', 'img_13', 'img_14', 'img_15', 'img_16', 'img_17', 'img_18', 'img_19', 'img_20', 'img_21', 'img_22', 'img_23', 'img_24', 'img_25', 'img_26', 'img_27', 'img_28', 'img_29', 'img_30', 'img_31', 'img_32', 'img_33', 'img_34', 'img_35', 'img_36', 'img_37', 'img_38', 'img_39', 'img_40', 'img_41', 'img_42', 'img_43', 'img_44', 'img_45', 'img_46', 'img_47', 'img_48', 'img_49', 'img_50', 'img_51', 'img_52', 'img_53', 'img_54', 'img_55', 'img_56', 'img_57', 'img_58', 'img_59', 'img_60', 'img_61', 'img_62', 'img_63', 'img_64', 'img_65', 'img_66', 'img_67', 'img_68', 'img_69', 'img_70', 'img_71', 'img_72', 'img_73', 'img_74', 'img_75', 'img_76', 'img_77', 'img_78', 'img_79', 'img_80', 'img_81', 'img_82', 'img_83', 'img_84', 'img_85', 'img_86', 'img_87', 'img_88', 'img_89', 'img_90', 'img_91', 'img_92', 'img_93', 'img_94', 'img_95', 'img_96', 'img_97', 'img_98', 'img_99']
['label_0

TypeError: __init__() got an unexpected keyword argument 'test_size'

## Visda2017 test set (assign labels)

In [14]:
import shutil
import os

data = 'data/visda2017/val'

for path, dns, fns in os.walk(data):
    for fn in fns:
        fn_split = fn.split('_')
        fn_split[1] = fn_split[1][:-4]
        new_fn = '_'.join(fn_split)
        shutil.move(f'{path}/{fn}', f'{path}/{new_fn}')